## Fu: calculate velocities

In [ ]:
import numpy as np
import pandas as pd
import scipy.stats as stats
import scanpy as sc
import anndata as ad
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.colors import ListedColormap
import scvelo as scv

### colormap for UMAP

In [ ]:
# list of HEX colors
hex_colors = ["#D90254", "#343AAD", "#04B2C6", "#73AF2A", "#FDB81E", "#5F3530", "#7F009E", "#0B7CE3",
              "#088775", "#C0D33C", "#FB8606", "#888888", "#582399", "#269CFC", "#39A240"]

# convert HEX colors to RGB
rgb_colors = [(int(color[1:3], 16)/255, int(color[3:5], 16)/255, int(color[5:7], 16)/255) for color in hex_colors]

# create colormap
cmap = ListedColormap(rgb_colors)

# create dictionary to assign colors to clusters
clusters = ["c01", "c02", "c03", "c04", "c05", "c06", "c07", "c08", "c09", "c10", "c11", "c12", "c13", "c14", "c15"]
umap_colors = {}
i = 0
for cluster in clusters:
    umap_colors[cluster] = cmap.colors[i]
    i+=1

### read in data

cellranger/velocyto files

In [ ]:
# adata files from cellranger
cellranger = sc.read_h5ad("../data/fu_cellranger.h5ad")

In [ ]:
# loom files from velocyto
ldata1 = scv.read('../velocyto/MJ001.loom', cache=True)
ldata2 = scv.read('../velocyto/MJ002.loom', cache=True) 
ldata3 = scv.read('../velocyto/MJ003.loom', cache=True) 
ldata5 = scv.read('../velocyto/MJ005.loom', cache=True)
ldata6 = scv.read('../velocyto/MJ006.loom', cache=True)
ldata7 = scv.read('../velocyto/MJ007.loom', cache=True) 
ldata8 = scv.read('../velocyto/MJ008.loom', cache=True) 
ldata9 = scv.read('../velocyto/MJ009.loom', cache=True) 
ldata16 = scv.read('../velocyto/MJ016.loom', cache=True) 
ldata17 = scv.read('../velocyto/MJ017.loom', cache=True) 
ldata18 = scv.read('../velocyto/MJ018.loom', cache=True) 
ldata19 = scv.read('../velocyto/MJ019.loom', cache=True)

In [ ]:
# rename barcodes
ldata1.obs_names = ldata1.obs_names.str.replace("x", "-1_1").str.replace("MJ001:", "")
ldata2.obs_names = ldata2.obs_names.str.replace("x", "-1_2").str.replace("MJ002:", "")
ldata3.obs_names = ldata3.obs_names.str.replace("x", "-1_3").str.replace("MJ003:", "")
ldata5.obs_names = ldata5.obs_names.str.replace("x", "-1_4").str.replace("MJ005:", "")
ldata6.obs_names = ldata6.obs_names.str.replace("x", "-1_5").str.replace("MJ006:", "")
ldata7.obs_names = ldata7.obs_names.str.replace("x", "-1_12").str.replace("MJ007:", "")
ldata8.obs_names = ldata8.obs_names.str.replace("x", "-1_6").str.replace("MJ008:", "")
ldata9.obs_names = ldata9.obs_names.str.replace("x", "-1_7").str.replace("MJ009:", "")
ldata16.obs_names = ldata16.obs_names.str.replace("x", "-1_8").str.replace("MJ016:", "")
ldata17.obs_names = ldata17.obs_names.str.replace("x", "-1_9").str.replace("MJ017:", "")
ldata18.obs_names = ldata18.obs_names.str.replace("x", "-1_10").str.replace("MJ018:", "")
ldata19.obs_names = ldata19.obs_names.str.replace("x", "-1_11").str.replace("MJ019:", "")

In [ ]:
# merge adata with loom file generated by velocyto
adata1 = scv.utils.merge(cellranger[cellranger.obs['id'] == 'MJ001'], ldata1)
adata2 = scv.utils.merge(cellranger[cellranger.obs['id'] == 'MJ002'], ldata2)
adata3 = scv.utils.merge(cellranger[cellranger.obs['id'] == 'MJ003'], ldata3)
adata5 = scv.utils.merge(cellranger[cellranger.obs['id'] == 'MJ005'], ldata5)
adata6 = scv.utils.merge(cellranger[cellranger.obs['id'] == 'MJ006'], ldata6)
adata7 = scv.utils.merge(cellranger[cellranger.obs['id'] == 'MJ007'], ldata7)
adata8 = scv.utils.merge(cellranger[cellranger.obs['id'] == 'MJ008'], ldata8)
adata9 = scv.utils.merge(cellranger[cellranger.obs['id'] == 'MJ009'], ldata9)
adata16 = scv.utils.merge(cellranger[cellranger.obs['id'] == 'MJ016'], ldata16)
adata17 = scv.utils.merge(cellranger[cellranger.obs['id'] == 'MJ017'], ldata17)
adata18 = scv.utils.merge(cellranger[cellranger.obs['id'] == 'MJ018'], ldata18)
adata19 = scv.utils.merge(cellranger[cellranger.obs['id'] == 'MJ019'], ldata19)

In [ ]:
# concatenate samples
velocyto = adata1.concatenate(adata2, adata3, adata5, adata6, adata7, adata8, adata9, adata16, adata17, adata18, adata19, index_unique=None)

alevin-fry file

In [ ]:
alevin = sc.read_h5ad("../data/fu_salmon.h5ad")

### compute velocities

In [ ]:
# preprocessing
scv.pp.filter_genes(velocyto)
scv.pp.normalize_per_cell(velocyto)

scv.pp.filter_genes(alevin)
scv.pp.normalize_per_cell(alevin)

# ensure that highly variable genes are the same
assert (velocyto.var["highly_variable"].sort_index()).equals(alevin.var["highly_variable"].sort_index()), "computing velocities on different genes"

In [ ]:
# compute neighborhood graph and moments for velocity estimation
scv.pp.moments(velocyto)
scv.pp.moments(alevin)

In [ ]:
# separate into 3 objects for 3 modes (deterministic, stochastic, dynamical)
velocyto_det = velocyto.copy()
velocyto_sto = velocyto.copy()
velocyto_dyn = velocyto.copy()

alevin_det = alevin.copy()
alevin_sto = alevin.copy()
alevin_dyn = alevin.copy()

#### model = deterministic

In [ ]:
# calculate velocities: deterministic (= steady state = velocyto)
scv.tl.velocity(velocyto_det, mode='deterministic', filter_genes=False)
scv.tl.velocity_graph(velocyto_det)

scv.tl.velocity(alevin_det, mode='deterministic', filter_genes=False)
scv.tl.velocity_graph(alevin_det)

In [ ]:
# embedding in UMAP
scv.pl.velocity_embedding_grid(velocyto_det, basis='umap', color="cluster", arrow_length = 2, palette=umap_colors,
                               legend_loc = 'right_margin', xlabel = "UMAP 1", ylabel = "UMAP 2")

scv.pl.velocity_embedding_grid(alevin_det, basis='umap', color="cluster", arrow_length = 2, palette=umap_colors,
                               legend_loc = 'right_margin', xlabel = "UMAP 1", ylabel = "UMAP 2")

#### model = stochastic

In [ ]:
# calculate velocities: stochastic 
scv.tl.velocity(velocyto_sto, mode='stochastic', filter_genes=False)
scv.tl.velocity_graph(velocyto_sto)

scv.tl.velocity(alevin_sto, mode='stochastic', filter_genes=False)
scv.tl.velocity_graph(alevin_sto)

In [ ]:
# embedding in UMAP
scv.pl.velocity_embedding_grid(velocyto_sto, basis='umap', color="cluster", arrow_length = 2, palette=umap_colors,
                               legend_loc = 'right_margin', xlabel = "UMAP 1", ylabel = "UMAP 2")

scv.pl.velocity_embedding_grid(alevin_sto, basis='umap', color="cluster", arrow_length = 2, palette=umap_colors,
                               legend_loc = 'right_margin', xlabel = "UMAP 1", ylabel = "UMAP 2")

#### model = dynamical

In [ ]:
# calculate velocities: dynamical 
scv.tl.recover_dynamics(velocyto_dyn)
scv.tl.velocity(velocyto_dyn, mode="dynamical", filter_genes=False)

scv.tl.recover_dynamics(alevin_dyn)
scv.tl.velocity(alevin_dyn, mode="dynamical", filter_genes=False)

In [ ]:
# velocity graph
scv.tl.velocity_graph(velocyto_dyn)

scv.tl.velocity_graph(alevin_dyn)

In [ ]:
# embedding in UMAP
scv.pl.velocity_embedding_grid(velocyto_dyn, basis='umap', color="cluster", arrow_length = 2, palette=umap_colors,
                               legend_loc = 'right_margin', xlabel = "UMAP 1", ylabel = "UMAP 2")

scv.pl.velocity_embedding_grid(alevin_dyn, basis='umap', color="cluster", arrow_length = 2, palette=umap_colors,
                               legend_loc = 'right_margin', xlabel = "UMAP 1", ylabel = "UMAP 2")

### save

In [ ]:
velocyto_sto.write_h5ad(filename="../data/fu_velocyto.h5ad")
alevin_sto.write_h5ad(filename="../data/fu_alevin.h5ad")